# Pairwise Experiments

### Setup

In [1]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = "" # API key is sensitive to share
os.environ["LANGSMITH_API_KEY"] = "" # API key is sensitive to share
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

### Task

Let's set up a new task! Here, we have a salesperson named Bob. Bob has a lot of deals, so he wants to summarize what happened in this deals based off of some meeting transcripts.

Bob is iterating on a few different prompts, that will give him nice, concise transcripts for his deals.

Bob has curated a dataset of his deal transcripts, let's go ahead and load that in. You can take a look at the dataset as well if you're curious! Note that this is not a golden dataset, there is no reference output here.

In [3]:
from langsmith import Client

client = Client()
dataset = client.clone_public_dataset(
  "https://smith.langchain.com/public/9078d2f1-7bef-4ba7-b795-210a17682ef9/d"
)

### Experiments

Now, let's run some experiments on this dataset using two different prompts. Let's add an evaluator that tries to score how good our summaries are!

In [4]:
from pydantic import BaseModel, Field
from openai import OpenAI

openai_client = OpenAI()

SUMMARIZATION_SYSTEM_PROMPT = """You are a judge, aiming to score how well a summary summarizes the content of a transcript"""

SUMMARIZATION_HUMAN_PROMPT = """
[The Meeting Transcript] {transcript}
[The Start of Summarization] {summary} [The End of Summarization]"""

class SummarizationScore(BaseModel):
    score: int = Field(description="""A score from 1-5 ranking how good the summarization is for the provided transcript, with 1 being a bad summary, and 5 being a great summary""")

def summary_score_evaluator(inputs: dict, outputs: dict) -> list:
    completion = openai_client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": SUMMARIZATION_SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": SUMMARIZATION_HUMAN_PROMPT.format(
                    transcript=inputs["transcript"],
                    summary=outputs.get("output", "N/A"),
                )}
        ],
        response_format=SummarizationScore,
    )

    summary_score = completion.choices[0].message.parsed.score
    return {"key": "summary_score", "score": summary_score}

First, we'll run our experiment with a good version of our prompt!

In [5]:
# Prompt One: Good Prompt!
def good_summarizer(inputs: dict):
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": f"Concisely summarize this meeting in 3 sentences. Make sure to include all of the important events. Meeting: {inputs['transcript']}"
            }
        ],
    )
    return response.choices[0].message.content

client.evaluate(
    good_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Good Summarizer"
)

View the evaluation results for experiment: 'Good Summarizer-533c6be7' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/5925bc48-4e25-427a-9231-dfdbd04387ec/compare?selectedSessions=4fe0a5c1-4cf8-4483-84e6-4d9673a2ae3a




0it [00:00, ?it/s]

,inputs.transcript,outputs.output,error,feedback.summary_score,execution_time,example_id,id
0,"Bob and Mr. Patel (CLOSED DEAL): Bob: Hello, M...",Bob successfully closed a deal with Mr. Patel ...,None,5,6.439287,4f803b9f-c605-4e41-99fc-3cb062d00587,9435432b-46d4-4084-a363-c4b1067eee80
1,"Bob and Ms. Thompson (NO DEAL): Bob: Hi, Ms. T...","In the meeting, Bob welcomed Ms. Thompson to F...",None,5,2.305715,55dc1cad-8516-4d80-b16c-634d8cacf72b,fd54edde-a7f2-4c70-b241-af9b7a135b9a
2,Bob and Mr. Johnson (CLOSED DEAL): Bob: Good m...,Bob warmly welcomed Mr. Johnson at Ford Motors...,None,5,2.571181,6211725a-f93b-4d98-8b09-a6c89e421078,2a855bd2-4bdb-4051-b33a-363e5ffd29ce
3,Bob and Mr. Carter (CLOSED DEAL): Bob: Welcome...,"In the meeting, Mr. Carter visited Bob to trad...",None,5,1.537278,6efb2329-d624-4468-beba-e41134069113,f71f6ef7-346e-41a4-a093-0dcae818641c
4,Bob and Ms. Nguyen (NO DEAL): Bob: Good aftern...,Bob and Ms. Nguyen discussed her interest in a...,None,5,1.350372,8d1c7d5f-ff17-4fc9-acf6-94bcdfce9fee,8d683ca3-cb57-4987-ac5a-cde2c4ca39ca


Now, we'll run an experiment with a worse version of our prompt, to highlight the difference.

In [6]:
# Prompt Two: Worse Prompt!
def bad_summarizer(inputs: dict):
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": f"Summarize this in one sentence. {inputs['transcript']}"
            }
        ],
    )
    return response.choices[0].message.content

client.evaluate(
    bad_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Bad Summarizer"
)

View the evaluation results for experiment: 'Bad Summarizer-aca042f5' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/5925bc48-4e25-427a-9231-dfdbd04387ec/compare?selectedSessions=74874f86-9efa-4759-b4ae-db3f3a9ba9c2




0it [00:00, ?it/s]

,inputs.transcript,outputs.output,error,feedback.summary_score,execution_time,example_id,id
0,"Bob and Mr. Patel (CLOSED DEAL): Bob: Hello, M...",Bob successfully closed a deal with Mr. Patel ...,None,5,1.049266,4f803b9f-c605-4e41-99fc-3cb062d00587,eaf1b928-46de-418e-a5bc-40cd4e01739e
1,"Bob and Ms. Thompson (NO DEAL): Bob: Hi, Ms. T...",Bob introduces Ms. Thompson to Ford's SUV opti...,None,5,1.085982,55dc1cad-8516-4d80-b16c-634d8cacf72b,dfc0ee08-df7e-49ec-ade0-74e0db651030
2,Bob and Mr. Johnson (CLOSED DEAL): Bob: Good m...,Bob successfully sold Mr. Johnson a Ford Explo...,None,5,1.401505,6211725a-f93b-4d98-8b09-a6c89e421078,ae4ec784-24b3-4637-8ef8-fb4e47537f46
3,Bob and Mr. Carter (CLOSED DEAL): Bob: Welcome...,Bob successfully facilitated a trade-in deal w...,None,5,1.403068,6efb2329-d624-4468-beba-e41134069113,e8fcb5d6-c7c6-4b51-9ff8-c6ff15d7f3a2
4,Bob and Ms. Nguyen (NO DEAL): Bob: Good aftern...,Bob unsuccessfully attempts to persuade Ms. Ng...,None,5,1.228144,8d1c7d5f-ff17-4fc9-acf6-94bcdfce9fee,3c3e5b40-dae4-40d6-ab46-022eeb14355e


### Pairwise Experiment

Let's define a function that will compare our two experiments. These are the fields that pairwise evaluator functions get access to:
- `inputs: dict`: A dictionary of the inputs corresponding to a single example in a dataset.
- `outputs: list[dict]`: A list of the dict outputs produced by each experiment on the given inputs.
- `reference_outputs: dict`: A dictionary of the reference outputs associated with the example, if available.
- `runs: list[Run]`: A list of the full Run objects generated by the experiments on the given example. Use this if you need access to intermediate steps or metadata about each run.
- `example: Example`: The full dataset Example, including the example inputs, outputs (if available), and metdata (if available).

First, let's give our LLM-as-Judge some instructions. In our case, we're just going to directly use LLM-as-judge to grade which of the summarizers is the most helpful.

It might be hard to grade our summarizers without a ground truth reference, but here, comparing different prompts head to head will give us a sense of which is better!

In [7]:
JUDGE_SYSTEM_PROMPT = """
Please act as an impartial judge and evaluate the quality of the summarizations provided by two AI summarizers to the meeting transcript below.
Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their summarizations.
Begin your evaluation by comparing the two summarizations and provide a short explanation.
Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision.
Do not favor certain names of the assistants.
Be as objective as possible. """

JUDGE_HUMAN_PROMPT = """
[The Meeting Transcript] {transcript}

[The Start of Assistant A's Summarization] {answer_a} [The End of Assistant A's Summarization]

[The Start of Assistant B's Summarization] {answer_b} [The End of Assistant B's Summarization]"""

Our function will take in an `inputs` dictionary, and a list of `outputs` dictionaries for the different experiments that we want to compare.

In [8]:
from pydantic import BaseModel, Field

class Preference(BaseModel):
    preference: int = Field(description="""1 if Assistant A answer is better based upon the factors above.
2 if Assistant B answer is better based upon the factors above.
Output 0 if it is a tie.""")

def ranked_preference(inputs: dict, outputs: list[dict]) -> list:
    completion = openai_client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": JUDGE_SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": JUDGE_HUMAN_PROMPT.format(
                    transcript=inputs["transcript"],
                    answer_a=outputs[0].get("output", "N/A"),
                    answer_b=outputs[1].get("output", "N/A")
                )}
        ],
        response_format=Preference,
    )

    preference_score = completion.choices[0].message.parsed.preference

    if preference_score == 1:
        scores = [1, 0]
    elif preference_score == 2:
        scores = [0, 1]
    else:
        scores = [0, 0]
    return scores

Now let's run our pairwise experiment with `evaluate()`

In [10]:
from langsmith import evaluate

evaluate(
    ("Good Summarizer-533c6be7", "Bad Summarizer-aca042f5"),  # TODO: Replace with the names/IDs of your experiments
    evaluators=[ranked_preference]
)

View the pairwise evaluation results at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/5925bc48-4e25-427a-9231-dfdbd04387ec/compare?selectedSessions=4fe0a5c1-4cf8-4483-84e6-4d9673a2ae3a%2C74874f86-9efa-4759-b4ae-db3f3a9ba9c2&comparativeExperiment=b6c07dc5-76c0-45d9-bb97-b100889a693c




  0%|          | 0/5 [00:00<?, ?it/s]